In [ ]:
%matplotlib notebook

# https://en.wikipedia.org/wiki/Bessel_filter
# https://www.recordingblogs.com/wiki/bessel-filter
# https://www.recordingblogs.com/wiki/bilinear-transformation
# https://tttapa.github.io/Pages/Mathematics/Systems-and-Control-Theory/Digital-filters/Discretization/Discretization-of-a-fourth-order-Butterworth-filter.html

import math
import operator
import sympy

import scipy.signal as sgn

import numpy as np
import matplotlib.pyplot as plt

s, p, g, z, Z = sympy.symbols('s p gamma z Z')

# p = s / omega_c

w0, wb, fc, fs = sympy.symbols('omega_0 omega_b F_c F_s')
# F_s sampling frequency
# F_c cutting frequency

# Order butterworth low pass with bilinear transform
## Butterworth polynomials

In [ ]:
def ButterworthPolynomial(n:int) :
    assert(1 <= n)
    u = [
        (p**2 - 2*p*sympy.cos(sympy.pi*(2*k+n-1)/(2*n)) + 1) for k in range(1, n // 2 + 1)
    ]
    if n % 2 != 0 : # polynom is odd
        u.append(p + 1)
    return sympy.prod(u)

In [ ]:
ButterworthPolynomial(4)

In [ ]:
sympy.N(ButterworthPolynomial(4))

## 4th order

In [ ]:
ButterworthPolynomial(4).expand().collect(p)

In [ ]:
str(ButterworthPolynomial(4).expand().collect(p))

Pour simplifier cette expression on pose $alpha$ et $beta$

In [ ]:
alpha, beta = sympy.symbols('alpha beta')

subs_alpha_beta = {
    'alpha': 2*sympy.sqrt((2 - sympy.sqrt(2))/4),
    'beta': 2*sympy.sqrt((2 + sympy.sqrt(2))/4),
}

B4a = p**4 + p**3*(alpha + beta) + p**2*(alpha*beta + 2) + p*(alpha + beta) + 1
B4a

In [ ]:
B4a.subs(subs_alpha_beta)

In [ ]:
B4d = B4a.subs({'p' : g * (z - 1) / (z + 1)}).together()
B4d

In [ ]:
H4d = (1 / B4d)
H4d

In [ ]:
H4d_num, H4d_den = H4d.as_numer_denom()

In [ ]:
sympy.Poly(H4d_num.expand().collect(z), z).all_coeffs()

In [ ]:
sympy.Poly(H4d_den.expand().collect(z), z).all_coeffs()

## 2nd order

In [ ]:
B2a = ButterworthPolynomial(2).expand().collect(p)

In [ ]:
B2d = B2a.subs({'p' : g * (z - 1) / (z + 1)}).together()
B2d

In [ ]:
H2d = (1 / B2d)
H2d

In [ ]:
H2d_num, H2d_den = H2d.as_numer_denom()

In [ ]:
sympy.Poly(H2d_num.expand().collect(z), z).all_coeffs()

In [ ]:
sympy.Poly(H2d_den.expand().collect(z), z).all_coeffs()

In [ ]:
str(sympy.Poly(H2d_den.expand().collect(z), z).all_coeffs())

## Direct form I (2nd order) with wrap

In [ ]:
def Butterworth_LowPass_2nd(fc:float, fs=1000) :
    wcd = math.tau * fc / fs
    assert(0.0 <= wcd < math.pi)
    
    g = 1 / math.tan(wcd / 2)
    
    b = [1, 2, 1]
    a = [g**2 + math.sqrt(2)*g + 1, 2 - 2*g**2, g**2 - math.sqrt(2)*g + 1]
    
    return [i/a[0] for i in b], [i/a[0] for i in a]

In [ ]:
BW_num, BW_den = Butterworth_LowPass_2nd(45.0, 360.0)
BW_num

In [ ]:
BW_den

In [ ]:
class IIR_direct_form_I_2nd_order() :
    
    order = 2
    
    def __init__(self, num, den, x_ini=0.0, y_ini=0.0) :
        self.x = [x_ini,] * (self.order + 1)
        self.y = [y_ini,] * (self.order + 1)
        
        self.b = num[:3]
        self.a = den[:3]
        
    def unwrap_360(self, x0, x1) :
        d = x0 - x1
        w = math.copysign(360.0, d) if abs(d) > 180.0 else 0.0
        return w
        
    def run(self, x0) :        
        b0, b1, b2 = self.b
        a0, a1, a2 = self.a
        
        self.x = [x0,] + self.x[:-1]
        # self.y = [0.0,] + self.y[:-1]
        
        x0, x1, x2 = self.x
        y1, y2 = self.y[:-1]
                
        y0 = b0 * x0 + b1 * x1 + b2 * x2 - (
            a1 * y1 + a2 * y1
        )
        
        # y0 = b0 * x0 + (1-b0) * y1
        
        self.y = [y0,] + self.y[:-1]
        
        # print(x0, self.x, self.y, y0)
        print(x0, self.x, y0)
                       
        return self.y[0]
    
    def run_wrapped_360(self, x0) :
        b0, b1, b2 = self.b
        a0, a1, a2 = self.a
        
        # for old elements, an unwrap is applied first
        w = self.unwrap_360(x0, self.x[0])
        self.x[1:] = [w + x for x in self.x[:-1]]
        self.y[1:] = [w + y for y in self.y[:-1]]
        
        self.x[0] = x0 % 360.0 # input are forced into [0.0,360.0[ to be sure
        
        x0, x1, x2 = self.x
        _, y1, y2 = self.y
        
        y0 = b0 * x0 + b1 * x1 + b2 * x2 - (
            a1 * y1 + a2 * y1
        )
        
        # y0 = (b0 * x0 + (1-b0) * y1)
        
        self.y[0] = y0
        
        # print(x0, self.x, '\n\t', w, self.y, y0, [y - 360.0 for y in self.y], y0 - 360.0)
        print(x0, self.x, y0)

        return self.y[0] % 360.0 # output are forced into [0.0, 360.0[

In [ ]:
BW = IIR_direct_form_I_2nd_order(BW_num, BW_den, x_ini=0.0, y_ini=0.0)

x_arr = np.array([0.0,] * 5 + [30.0,] * 10 + [360.0 - 30.0,] * 10 + [30.0,] * 10)
y_arr = np.array([BW.run_wrapped_360(x) for x in x_arr])

In [ ]:
plt.figure()
plt.plot(x_arr)
plt.plot(y_arr, '+-')
plt.grid()
plt.show()

In [ ]:
BW = IIR_direct_form_I_2nd_order(BW_num, BW_den, x_ini=0.0, y_ini=0.0)

x_arr = np.array([0.0,] * 5 + [30.0,] * 10 + [-30.0,] * 10 + [30.0,] * 10)
y_arr = np.array([BW.run(x) for x in x_arr])

In [ ]:
plt.figure()
plt.plot(x_arr)
plt.plot(y_arr, '+-')
plt.grid()
plt.show()